In [ ]:
import pandas as pd
from collections import namedtuple

KEY = "Response ID"

# Columns names when a rename is performed
TITLE = "Title"
FOOTAGE = "Footage Used"
AUDIO = "Music/Audio Used"
AUDIO_ARTIST = "Music/Audio Artists Used"
LINK = "Download Link"
DEBUT_DATE = "Debut Date"
NOTES = "Additional Notes"
KIND = "Category"

# Category Types
BONUS = "Bonus"
LIMITED = "Limited"
STANDARD = "Standard"

# Category column names
BonusCategoryColumns = namedtuple('BonusCategoryColumns', ['title', 'footage', 'audio', 'audio_artist', 'link', 'debut_date', 'notes'])
MainCategoryColumns = namedtuple('MainCategoryColumns', ['title', 'footage', 'audio', 'audio_artist', 'link', 'debut_date', 'notes', 'kind'])

def getLimitedCategoryColumns():
    title            = f"{LIMITED} Category AMV - Title"
    footage          = f"{LIMITED} Category AMV - List of Footage Used"
    audio            = f"{LIMITED} Category AMV - List of Music/Audio Used"
    audio_artist     = f"{LIMITED} Category AMV - List of Music/Audio Artists Used"
    link             = f"{LIMITED} Category AMV - Download Link Please enter a Dropbox, Sendspace, Google Drive, or Amazon AWS link to your video here."
    debut_date       = f"{LIMITED} Category AMV - Debut Date - When was this AMV first exhibited publicly or viewable online? If it is debuting at AX 2022, enter July 1st, 2022."
    notes            = f"{LIMITED} Category AMV - Additional Notes"
    kind             = f"What kind of {LIMITED} Category AMV are you submitting?"	
    return MainCategoryColumns(title, footage, audio, audio_artist, link, debut_date, notes, kind)

def getCategoryColumns(category_type, number=1):
    if(category_type == LIMITED):
        return getLimitedCategoryColumns()

    title            = f"{category_type} Category AMV {number} - Title"
    footage          = f"{category_type} Category AMV {number} - List of Footage Used"
    audio            = f"{category_type} Category AMV {number} - List of Music/Audio Used"
    audio_artist     = f"{category_type} Category AMV {number} - List of Music/Audio Artists Used"
    link             = f"{category_type} Category AMV {number} - Download Link Please enter a Dropbox, Sendspace, Google Drive, or Amazon AWS link to your video here."
    debut_date       = f"{category_type} Category AMV {number} - Debut Date - When was this AMV first exhibited publicly or viewable online? If it is debuting at AX 2022, enter July 1st, 2022."
    notes            = f"{category_type} Category AMV {number} - Additional Notes"

    if(category_type == BONUS):
        return BonusCategoryColumns(title, footage, audio, audio_artist, link, debut_date, notes)
    else:
        kind = f"What kind of {category_type} Category AMV {number} are you submitting?"	
        return MainCategoryColumns(title, footage, audio, audio_artist, link, debut_date, notes, kind)

BC_1_COLS = getCategoryColumns(BONUS, "#1")
BC_2_COLS = getCategoryColumns(BONUS, "#2")
BC_3_COLS = getCategoryColumns(BONUS, "#3")

SC_1_COLS = getCategoryColumns(STANDARD, "#1")
SC_2_COLS = getCategoryColumns(STANDARD, "#2")
SC_3_COLS = getCategoryColumns(STANDARD, "#3")

LC_COLS = getCategoryColumns(LIMITED)

In [17]:
def bonusCategoryRename(og_col_names):
    """
    Returns a dictionary used to rename the columns
    """
    renames = {
        og_col_names.title : TITLE,
        og_col_names.footage : FOOTAGE,
        og_col_names.audio : AUDIO,
        og_col_names.audio_artist : AUDIO_ARTIST,
        og_col_names.link : LINK,
        og_col_names.debut_date : DEBUT_DATE,
        og_col_names.notes : NOTES
    }
    return renames

def mainCategoryRename(og_col_names):
    """
    
    """
    renames = bonusCategoryRename(og_col_names)
    renames[og_col_names.kind] = KIND
    return renames

In [ ]:
# Importing data
amv_df = pd.read_excel('./test.xlsx')

In [ ]:
def columnList(columns):
    column_list = list(columns)
    column_list.insert(0, KEY)
    return column_list

In [ ]:
BC_1_COL_LIST = columnList(BC_1_COLS)
BC_2_COL_LIST = columnList(BC_2_COLS)
BC_3_COL_LIST = columnList(BC_3_COLS)

In [ ]:
# Gather each subsection of Bonus Columns
bonus_df_1 = amv_df[BC_1_COL_LIST]
bonus_df_2 = amv_df[BC_2_COL_LIST]
bonus_df_3 = amv_df[BC_3_COL_LIST]

# Get the renames for each of the columns
bonus_rename_1 = bonusCategoryRename(BC_1_COLS)
bonus_rename_2 = bonusCategoryRename(BC_2_COLS)
bonus_rename_3 = bonusCategoryRename(BC_3_COLS)

# Perform column rename on each dataframe
# Each df will have the same set of names for their columns
bonus_df_1 = bonus_df_1.rename(columns=bonus_rename_1)
bonus_df_2 = bonus_df_2.rename(columns=bonus_rename_2)
bonus_df_3 = bonus_df_3.rename(columns=bonus_rename_3)

In [ ]:
# Concatenate the columns
bonus_df = pd.concat([bonus_df_1, bonus_df_2, bonus_df_3])
bonus_df = bonus_df.dropna(subset=[TITLE])

In [ ]:
result_path = str('./' + 'commercial_movie_trailer_parodies' + '.xlsx') 
with pd.ExcelWriter(result_path, engine='xlsxwriter') as writer:
    bonus_df.to_excel(writer, index=False)

### Limited Category Breakdown

In [ ]:
# Limited Categories
ACTION = "Action"
SENTIMENTS = "Sentiments"
DRAMA = "Drama"

In [ ]:
LC_COL_LIST = columnList(LC_COLS)
LC_KIND = LC_COLS.kind

lc_df = amv_df[LC_COL_LIST]

# Break up into the 3 categories that Limited Contains
action_df = lc_df[lc_df[LC_KIND] == ACTION]
sentiments_df = lc_df[lc_df[LC_KIND] == SENTIMENTS]
drama_df = lc_df[lc_df[LC_KIND] == DRAMA]

# Remove Kind column as the type of information has already been used to 
# seperate the data into seperate dataframes. 
action_df = action_df.drop([LC_KIND], axis=1)
sentiments_df = sentiments_df.drop([LC_KIND], axis=1)
drama_df = drama_df.drop([LC_KIND], axis=1)

In [ ]:
lc_writer = pd.ExcelWriter('limited_categories.xlsx', engine='xlsxwriter')

# Write each Limited Category dataframe to a specific sheet
action_df.to_excel(lc_writer, sheet_name=ACTION)
sentiments_df.to_excel(lc_writer, sheet_name=SENTIMENTS)
drama_df.to_excel(lc_writer, sheet_name=DRAMA)

# Close the Pandas Excel writer and output the Excel file
lc_writer.save()

### Standard Category

In [18]:
# Standard Categories
FUN_AND_PLAY = "Fun and Play"
COMEDY_AND_HUMOR = "Comedy and Humor"
RHYTHM_AND_BEAT = "Rhythm and Beat"

In [19]:
SC_1_COL_LIST = columnList(SC_1_COLS)
SC_2_COL_LIST = columnList(SC_2_COLS)
SC_3_COL_LIST = columnList(SC_3_COLS)

In [20]:
# Gather each subsection of Standard Category Columns
sc_df_1 = amv_df[SC_1_COL_LIST]
sc_df_2 = amv_df[SC_2_COL_LIST]
sc_df_3 = amv_df[SC_3_COL_LIST]

# Get the renames for each of the columns
sc_rename_1 = mainCategoryRename(SC_1_COLS)
sc_rename_2 = mainCategoryRename(SC_2_COLS)
sc_rename_3 = mainCategoryRename(SC_3_COLS)

# Perform column rename on each dataframe
# Each df will have the same set of names for their columns
sc_df_1 = sc_df_1.rename(columns=sc_rename_1)
sc_df_2 = sc_df_2.rename(columns=sc_rename_2)
sc_df_3 = sc_df_3.rename(columns=sc_rename_3)

In [21]:
# Concatenate the columns
sc_df = pd.concat([sc_df_1, sc_df_2, sc_df_3])
sc_df = sc_df.dropna(subset=[TITLE])

Seperate into the different kinds of AMV's

In [30]:
# Break up into the 3 categories that Limited Contains
fun_and_play_df = sc_df[sc_df[KIND] == FUN_AND_PLAY]
comedy_and_humor_df = sc_df[sc_df[KIND] == COMEDY_AND_HUMOR]
rhythm_and_beat_df = sc_df[sc_df[KIND] == RHYTHM_AND_BEAT]

# Remove Kind column as the type of information has already been used to 
# seperate the data into seperate dataframes. 
fun_and_play_df = fun_and_play_df.drop([KIND], axis=1)
comedy_and_humor_df = comedy_and_humor_df.drop([KIND], axis=1)
rhythm_and_beat_df = rhythm_and_beat_df.drop([KIND], axis=1)

In [ ]:
sc_writer = pd.ExcelWriter('standard_categories.xlsx', engine='xlsxwriter')

# Write each Standard Category dataframe to a specific sheet
fun_and_play_df.to_excel(sc_writer, sheet_name=FUN_AND_PLAY)
comedy_and_humor_df.to_excel(sc_writer, sheet_name=COMEDY_AND_HUMOR)
rhythm_and_beat_df.to_excel(sc_writer, sheet_name=RHYTHM_AND_BEAT)

# Close the Pandas Excel writer and output the Excel file
sc_writer.save()